In [20]:
# import 
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader

# Create a fc net
class NN(nn.Module):
    def __init__(self, input_size, class_num):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, class_num)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    # set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# hyperparameter
learning_rate = 0.001
input_size = 784
class_num = 10
num_epochs = 2
batch_size = 64

# data load
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = NN(input_size=input_size, class_num=class_num).to(device)

# loss and optimizers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # if cuda available
        data = data.to(device=device)
        targets = targets.to(device=device)
        # reshape
        data = data.reshape(data.shape[0], -1)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient descent or Adam Step
        optimizer.step()
        
# check accuracy
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Check accuracy on train dataset")
    else:
        print("Check accuracy on test dataset")

    num_correct = 0
    num_sample = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_sample += predictions.size(0)
        print(f"Got {num_correct} / {num_sample} with accuracy {float(num_correct)/float(num_sample)*100:.2f}")
    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Check accuracy on train dataset
Got 56984 / 60000 with accuracy 94.97
Check accuracy on test dataset
Got 9480 / 10000 with accuracy 94.80
